In [2]:
import numpy as np
import matplotlib.pyplot as plt

In [3]:
import numpy as np
from ctypes import c_double, c_void_p, POINTER, cdll, c_int, addressof, c_char_p, c_char
from numpy.ctypeslib import ndpointer

In [4]:
# Load appropriate LAPACK library
#liblapack = cdll.LoadLibrary("/usr/local/lib/liblapack.dylib")
liblapack = cdll.LoadLibrary("/usr/local/Cellar/lapack/3.12.0/lib/liblapack.3.12.0.dylib")


# SciPy linalg lapack

In [16]:
import numpy as _np
from scipy.linalg.blas import _get_funcs, _memoize_get_funcs # from .blas import _get_funcs, _memoize_get_funcs
from scipy.linalg import _flapack
from re import compile as regex_compile
try:
    from scipy.linalg import _clapack
except ImportError:
    _clapack = None

try:
    from scipy.linalg import _flapack_64
    HAS_ILP64 = True
except ImportError:
    HAS_ILP64 = False
    _flapack_64 = None


In [17]:


# Expose all functions (only flapack --- clapack is an implementation detail)
empty_module = None
from scipy.linalg._flapack import *  # noqa: E402, F403
del empty_module

__all__ = ['get_lapack_funcs']

# some convenience alias for complex functions
_lapack_alias = {
    'corghr': 'cunghr', 'zorghr': 'zunghr',
    'corghr_lwork': 'cunghr_lwork', 'zorghr_lwork': 'zunghr_lwork',
    'corgqr': 'cungqr', 'zorgqr': 'zungqr',
    'cormqr': 'cunmqr', 'zormqr': 'zunmqr',
    'corgrq': 'cungrq', 'zorgrq': 'zungrq',
}


# Place guards against docstring rendering issues with special characters
p1 = regex_compile(r'with bounds (?P<b>.*?)( and (?P<s>.*?) storage){0,1}\n')
p2 = regex_compile(r'Default: (?P<d>.*?)\n')


def backtickrepl(m):
    if m.group('s'):
        return ('with bounds ``{}`` with ``{}`` storage\n'
                ''.format(m.group('b'), m.group('s')))
    else:
        return 'with bounds ``{}``\n'.format(m.group('b'))


for routine in [ssyevr, dsyevr, cheevr, zheevr,
                ssyevx, dsyevx, cheevx, zheevx,
                ssygvd, dsygvd, chegvd, zhegvd]:
    if routine.__doc__:
        routine.__doc__ = p1.sub(backtickrepl, routine.__doc__)
        routine.__doc__ = p2.sub('Default ``\\1``\n', routine.__doc__)
    else:
        continue

del regex_compile, p1, p2, backtickrepl


@_memoize_get_funcs
def get_lapack_funcs(names, arrays=(), dtype=None, ilp64=False):
    """Return available LAPACK function objects from names.

    Arrays are used to determine the optimal prefix of LAPACK routines.

    Parameters
    ----------
    names : str or sequence of str
        Name(s) of LAPACK functions without type prefix.

    arrays : sequence of ndarrays, optional
        Arrays can be given to determine optimal prefix of LAPACK
        routines. If not given, double-precision routines will be
        used, otherwise the most generic type in arrays will be used.

    dtype : str or dtype, optional
        Data-type specifier. Not used if `arrays` is non-empty.

    ilp64 : {True, False, 'preferred'}, optional
        Whether to return ILP64 routine variant.
        Choosing 'preferred' returns ILP64 routine if available, and
        otherwise the 32-bit routine. Default: False

    Returns
    -------
    funcs : list
        List containing the found function(s).

    Notes
    -----
    This routine automatically chooses between Fortran/C
    interfaces. Fortran code is used whenever possible for arrays with
    column major order. In all other cases, C code is preferred.

    In LAPACK, the naming convention is that all functions start with a
    type prefix, which depends on the type of the principal
    matrix. These can be one of {'s', 'd', 'c', 'z'} for the NumPy
    types {float32, float64, complex64, complex128} respectively, and
    are stored in attribute ``typecode`` of the returned functions.

    Examples
    --------
    Suppose we would like to use '?lange' routine which computes the selected
    norm of an array. We pass our array in order to get the correct 'lange'
    flavor.

    >>> import numpy as np
    >>> import scipy.linalg as LA
    >>> rng = np.random.default_rng()

    >>> a = rng.random((3,2))
    >>> x_lange = LA.get_lapack_funcs('lange', (a,))
    >>> x_lange.typecode
    'd'
    >>> x_lange = LA.get_lapack_funcs('lange',(a*1j,))
    >>> x_lange.typecode
    'z'

    Several LAPACK routines work best when its internal WORK array has
    the optimal size (big enough for fast computation and small enough to
    avoid waste of memory). This size is determined also by a dedicated query
    to the function which is often wrapped as a standalone function and
    commonly denoted as ``###_lwork``. Below is an example for ``?sysv``

    >>> a = rng.random((1000, 1000))
    >>> b = rng.random((1000, 1)) * 1j
    >>> # We pick up zsysv and zsysv_lwork due to b array
    ... xsysv, xlwork = LA.get_lapack_funcs(('sysv', 'sysv_lwork'), (a, b))
    >>> opt_lwork, _ = xlwork(a.shape[0])  # returns a complex for 'z' prefix
    >>> udut, ipiv, x, info = xsysv(a, b, lwork=int(opt_lwork.real))

    """
    if isinstance(ilp64, str):
        if ilp64 == 'preferred':
            ilp64 = HAS_ILP64
        else:
            raise ValueError("Invalid value for 'ilp64'")

    if not ilp64:
        print("this one)")
        return _get_funcs(names, arrays, dtype,
                          "LAPACK", _flapack, _clapack,
                          "flapack", "clapack", _lapack_alias,
                          ilp64=False)
    else:
        if not HAS_ILP64:
            raise RuntimeError("LAPACK ILP64 routine requested, but Scipy "
                               "compiled only with 32-bit BLAS")
        return _get_funcs(names, arrays, dtype,
                          "LAPACK", _flapack_64, None,
                          "flapack_64", None, _lapack_alias,
                          ilp64=True)


_int32_max = _np.iinfo(_np.int32).max
_int64_max = _np.iinfo(_np.int64).max


def _compute_lwork(routine, *args, **kwargs):
    """
    Round floating-point lwork returned by lapack to integer.

    Several LAPACK routines compute optimal values for LWORK, which
    they return in a floating-point variable. However, for large
    values of LWORK, single-precision floating point is not sufficient
    to hold the exact value --- some LAPACK versions (<= 3.5.0 at
    least) truncate the returned integer to single precision and in
    some cases this can be smaller than the required value.

    Examples
    --------
    >>> from scipy.linalg import lapack
    >>> n = 5000
    >>> s_r, s_lw = lapack.get_lapack_funcs(('sysvx', 'sysvx_lwork'))
    >>> lwork = lapack._compute_lwork(s_lw, n)
    >>> lwork
    32000

    """
    dtype = getattr(routine, 'dtype', None)
    int_dtype = getattr(routine, 'int_dtype', None)
    ret = routine(*args, **kwargs)
    if ret[-1] != 0:
        raise ValueError("Internal work array size computation failed: "
                         "%d" % (ret[-1],))

    if len(ret) == 2:
        return _check_work_float(ret[0].real, dtype, int_dtype)
    else:
        return tuple(_check_work_float(x.real, dtype, int_dtype)
                     for x in ret[:-1])


def _check_work_float(value, dtype, int_dtype):
    """
    Convert LAPACK-returned work array size float to integer,
    carefully for single-precision types.
    """

    if dtype == _np.float32 or dtype == _np.complex64:
        # Single-precision routine -- take next fp value to work
        # around possible truncation in LAPACK code
        value = _np.nextafter(value, _np.inf, dtype=_np.float32)

    value = int(value)
    if int_dtype.itemsize == 4:
        if value < 0 or value > _int32_max:
            raise ValueError("Too large work array required -- computation "
                             "cannot be performed with standard 32-bit"
                             " LAPACK.")
    elif int_dtype.itemsize == 8:
        if value < 0 or value > _int64_max:
            raise ValueError("Too large work array required -- computation"
                             " cannot be performed with standard 64-bit"
                             " LAPACK.")
    return value

In [18]:
get_lapack_funcs("gesvd")

this one)


<fortran object>

In [20]:
get_lapack_funcs("dggsvd")

this one)


ValueError: LAPACK function ddggsvd could not be found

# How scipy SVD works

In [ ]:
"""SVD decomposition functions."""
import numpy
from numpy import zeros, r_, diag, dot, arccos, arcsin, where, clip

# Local imports.
from ._misc import LinAlgError, _datacopied
from .lapack import get_lapack_funcs, _compute_lwork
from ._decomp import _asarray_validated

__all__ = ['svd', 'svdvals', 'diagsvd', 'orth', 'subspace_angles', 'null_space']


def svd(a, full_matrices=True, compute_uv=True, overwrite_a=False,
        check_finite=True, lapack_driver='gesdd'):
    """
    Singular Value Decomposition.

    Factorizes the matrix `a` into two unitary matrices ``U`` and ``Vh``, and
    a 1-D array ``s`` of singular values (real, non-negative) such that
    ``a == U @ S @ Vh``, where ``S`` is a suitably shaped matrix of zeros with
    main diagonal ``s``.

    Parameters
    ----------
    a : (M, N) array_like
        Matrix to decompose.
    full_matrices : bool, optional
        If True (default), `U` and `Vh` are of shape ``(M, M)``, ``(N, N)``.
        If False, the shapes are ``(M, K)`` and ``(K, N)``, where
        ``K = min(M, N)``.
    compute_uv : bool, optional
        Whether to compute also ``U`` and ``Vh`` in addition to ``s``.
        Default is True.
    overwrite_a : bool, optional
        Whether to overwrite `a`; may improve performance.
        Default is False.
    check_finite : bool, optional
        Whether to check that the input matrix contains only finite numbers.
        Disabling may give a performance gain, but may result in problems
        (crashes, non-termination) if the inputs do contain infinities or NaNs.
    lapack_driver : {'gesdd', 'gesvd'}, optional
        Whether to use the more efficient divide-and-conquer approach
        (``'gesdd'``) or general rectangular approach (``'gesvd'``)
        to compute the SVD. MATLAB and Octave use the ``'gesvd'`` approach.
        Default is ``'gesdd'``.

        .. versionadded:: 0.18

    Returns
    -------
    U : ndarray
        Unitary matrix having left singular vectors as columns.
        Of shape ``(M, M)`` or ``(M, K)``, depending on `full_matrices`.
    s : ndarray
        The singular values, sorted in non-increasing order.
        Of shape (K,), with ``K = min(M, N)``.
    Vh : ndarray
        Unitary matrix having right singular vectors as rows.
        Of shape ``(N, N)`` or ``(K, N)`` depending on `full_matrices`.

    For ``compute_uv=False``, only ``s`` is returned.

    Raises
    ------
    LinAlgError
        If SVD computation does not converge.

    See Also
    --------
    svdvals : Compute singular values of a matrix.
    diagsvd : Construct the Sigma matrix, given the vector s.

    Examples
    --------
    >>> import numpy as np
    >>> from scipy import linalg
    >>> rng = np.random.default_rng()
    >>> m, n = 9, 6
    >>> a = rng.standard_normal((m, n)) + 1.j*rng.standard_normal((m, n))
    >>> U, s, Vh = linalg.svd(a)
    >>> U.shape,  s.shape, Vh.shape
    ((9, 9), (6,), (6, 6))

    Reconstruct the original matrix from the decomposition:

    >>> sigma = np.zeros((m, n))
    >>> for i in range(min(m, n)):
    ...     sigma[i, i] = s[i]
    >>> a1 = np.dot(U, np.dot(sigma, Vh))
    >>> np.allclose(a, a1)
    True

    Alternatively, use ``full_matrices=False`` (notice that the shape of
    ``U`` is then ``(m, n)`` instead of ``(m, m)``):

    >>> U, s, Vh = linalg.svd(a, full_matrices=False)
    >>> U.shape, s.shape, Vh.shape
    ((9, 6), (6,), (6, 6))
    >>> S = np.diag(s)
    >>> np.allclose(a, np.dot(U, np.dot(S, Vh)))
    True

    >>> s2 = linalg.svd(a, compute_uv=False)
    >>> np.allclose(s, s2)
    True

    """
    a1 = _asarray_validated(a, check_finite=check_finite)
    if len(a1.shape) != 2:
        raise ValueError('expected matrix')
    m, n = a1.shape
    overwrite_a = overwrite_a or (_datacopied(a1, a))

    if not isinstance(lapack_driver, str):
        raise TypeError('lapack_driver must be a string')
    if lapack_driver not in ('gesdd', 'gesvd'):
        raise ValueError('lapack_driver must be "gesdd" or "gesvd", not "%s"'
                         % (lapack_driver,))
    funcs = (lapack_driver, lapack_driver + '_lwork')
    gesXd, gesXd_lwork = get_lapack_funcs(funcs, (a1,), ilp64='preferred')

    # compute optimal lwork
    lwork = _compute_lwork(gesXd_lwork, a1.shape[0], a1.shape[1],
                           compute_uv=compute_uv, full_matrices=full_matrices)

    # perform decomposition
    u, s, v, info = gesXd(a1, compute_uv=compute_uv, lwork=lwork,
                          full_matrices=full_matrices, overwrite_a=overwrite_a)

    if info > 0:
        raise LinAlgError("SVD did not converge")
    if info < 0:
        raise ValueError('illegal value in %dth argument of internal gesdd'
                         % -info)
    if compute_uv:
        return u, s, v
    else:
        return s

# OLD

In [4]:

# Define function for calling DGGSVD
def dgsvd(A, jobu="A", jobvt="A"):
  """
  Performs GSVD using LAPACK DGGSVD routine.

  Args:
    A (numpy.ndarray): Input matrix A (double precision).


  Returns:
    u (numpy.ndarray): Singular vectors of A.
    s (numpy.ndarray): Singular values.
    v (numpy.ndarray): Singular vectors of B.
  """

  ###  Handle inputs
  m, n = A.shape

  valid_jobs = ["A", "S", "O", "N"]
  assert jobu in valid_jobs, "invalid choice for jobu."
  assert jobvt in valid_jobs, "invalid choice for jobvt."
  assert not ((jobu == "O") and (jobvt == "O")), "jobu and jobvt cannot both be O."

  lda = m
  assert lda >= max([1, m]), "invalid choice for LDA"

  if jobu == "A":
    u_shape = (m,n)
    ldu = m
  elif jobu == "S":
    u_shape = (m, min([m,n]))
    ldu = m
  else:
    # u not referenced in this case
    u_shape = (1,1)
    ldu = 1

  if jobvt == "A":
    vt_shape = (n,n)
    ldvt = n
  elif jobvt == "S":
    #vt_shape = (min([m,n]), n)
    vt_shape = (n, min([m,n]))
    ldvt = min([m,n])
  else:
    # vt not referenced in this case
    vt_shape = (1,1)
    ldvt = 1


  lwork = -1

  jobu = jobu.encode("utf-8")
  jobvt = jobvt.encode("utf-8")



  # Convert A to C-compatible data structures
  a = A.ctypes.data_as(POINTER(c_double))
  #a_data = A.copy().ctypes.cast(POINTER(c_double), strides=False)
  #a_data = np.ctypes.data_as(a)

  # Allocate memory for output arrays
  s = np.empty( (min([m,n]),), dtype=np.double)
  #a = np.empty( (lda, n), dtype=np.double)
  u = np.empty( u_shape, dtype=np.double )
  vt = np.empty( vt_shape, dtype=np.double )
  work = np.empty( (100,), dtype=np.double )
  info = 0


  dgesvd_ = liblapack.dgesvd_
  dgesvd_.restype = None
  dgesvd_.argtypes = [
    c_char_p,
    c_char_p,
    c_int, 
    c_int,
    ndpointer(c_double, flags="C_CONTIGUOUS"),
    c_int,
    ndpointer(c_double, flags="C_CONTIGUOUS"),
    ndpointer(c_double, flags="C_CONTIGUOUS"),
    c_int, 
    ndpointer(c_double, flags="C_CONTIGUOUS"),
    c_int,
    ndpointer(c_double),
    c_int,
    c_int
  ]

# propagate = lib.propagate
# propagate.restype = None
# propagate.argtypes = [ndpointer(ctypes.c_float, flags="C_CONTIGUOUS"),
#                       ndpointer(ctypes.c_float, flags="C_CONTIGUOUS"),
#                       ndpointer(ctypes.c_float, flags="C_CONTIGUOUS"),
#                       ndpointer(ctypes.c_float, flags="C_CONTIGUOUS"),
#                       ctypes.c_float, ctypes.c_int, ctypes.c_int]


  return dgesvd_(jobu, jobvt, m, n, A, lda, s, u, ldu, vt, ldvt, work, 100, info )


  # liblapack.dgesvd_(c_char_p(jobu.encode("utf-8")),  # JOBU
  #                   c_char_p(jobvt.encode("utf-8")),  # JOBVT
  #                   c_int(m),  # M
  #                   c_int(n), # N
  #                   a,  # A
  #                   c_int(lda),  # LDA
  #                   s,  # S
  #                    u, # u
  #                     c_int(ldu), # ldu
  #                     vt, # vt
  #                     c_int(ldvt) # ldvt
  #                     None, # work
  #                     lwork, #lwork
  #                     c_int(info), # Info  
  #                     )  

  # if liblapack.dgesvd_.err == 1:
  #   raise Exception("DGGSVD error: The algorithm did not converge.")

  # return u, s, vt

In [5]:
c_char("A".encode("utf-8"))

c_char(b'A')

In [ ]:

# Example usage

a = np.random.rand(5,2).astype(np.double) 

res = dgsvd(a, jobu="N", jobvt="N")

print("U:", u)
print("S:", s)
print("V:", v)
print("Q:", q)

: 

In [ ]:
np.random.rand(5,3).astype(np.double)

array([[0.77527774, 0.87995414, 0.06275372],
       [0.05673105, 0.70302994, 0.23901943],
       [0.7829233 , 0.99197642, 0.13926028],
       [0.2499869 , 0.4925873 , 0.06164146],
       [0.56281278, 0.90729101, 0.800618  ]])

In [18]:
np.__version__

'1.21.4'

In [ ]:
# import numpy as np
# from ctypes import c_double, c_void_p, POINTER, cdll

# # Load appropriate LAPACK library
# liblapack = cdll.LoadLibrary("/usr/local/lib/liblapack.dylib")

# import numpy as np
# from ctypes import c_double, c_void_p, POINTER, cdll

# Load appropriate LAPACK library
# liblapack = cdll.LoadLibrary("/usr/local/lib/liblapack.dylib")

# Define function for calling DGGSVD
def dgsvd(a, b):
  """
  Performs GSVD using LAPACK DGGSVD routine.

  Args:
    a (numpy.ndarray): Input matrix A (double precision).
    b (numpy.ndarray): Input matrix B (double precision).

  Returns:
    u (numpy.ndarray): Singular vectors of A.
    s (numpy.ndarray): Singular values.
    v (numpy.ndarray): Singular vectors of B.
    q (numpy.ndarray): Q matrix.
  """
  n = a.shape[0]
  lda = a.shape[1]
  ldb = b.shape[1]

  # Allocate memory for output arrays
  u = np.empty((n, n), dtype=np.double)
  s = np.empty(min(n, lda), dtype=np.double)
  v = np.empty((lda, lda), dtype=np.double)
  q = np.empty((ldb, ldb), dtype=np.double)

  # Access underlying data buffer with np.ctypeslib
  #u_data = np.ctypeslib.as_array(u).ctypes.data_as(c_double * lda)
  u_data = np.ctypeslib.as_array(u).ctypes.data_as(POINTER(c_double))

  # Convert NumPy arrays to C-compatible data structures
  a_data = a.ctypes.data_as(POINTER(c_double))
  b_data = b.ctypes.data_as(POINTER(c_double))

  # Call DGGSVD routine
  liblapack.dgesvd_(c_void_p(b"N"), c_void_p(b"N"), c_int(n), c_int(lda),
                     c_int(ldb), a_data, c_int(lda), s.ctypes.data_as(POINTER(c_double)),
                     u_data, c_int(lda), v.ctypes.data_as(POINTER(c_double)), c_int(lda),
                     q.ctypes.data_as(POINTER(c_double)), c_int(ldb))

  # Check for errors (example)
  if liblapack.dgesvd_.err == 1:
    raise Exception("DGGSVD error: The algorithm did not converge.")

  return u, s, v, q

# Example usage
a = np.random.rand(5, 3)
b = np.random.rand(5, 2)

n = a.shape[0]
u = np.empty((n, n), dtype=np.double)
u, s, v, q = dgsvd(a, b)

print("U:", u)
print("S:", s)
print("V:", v)
print("Q:", q)

TypeError: cannot be converted to pointer

In [ ]:
# Example usage
a = np.random.rand(5, 3)
b = np.random.rand(5, 2)

n = a.shape[0]
lda = a.shape[1]
ldb = b.shape[1]

# Allocate memory for output arrays
u = np.empty((n, n), dtype=np.double)
s = np.empty(min(n, lda), dtype=np.double)
v = np.empty((lda, lda), dtype=np.double)
q = np.empty((ldb, ldb), dtype=np.double)

# Access underlying data buffer with np.ctypeslib
#u_data = np.ctypeslib.as_array(u).ctypes.data_as(c_double * lda)
u_data = np.ctypeslib.as_array(u).ctypes.data_as(POINTER(c_double))

# Convert NumPy arrays to C-compatible data structures
a_data = a.ctypes.data_as(POINTER(c_double))
b_data = b.ctypes.data_as(POINTER(c_double))

In [ ]:
c_void_p(b"N")

TypeError: cannot be converted to pointer

In [ ]:
import ctypes

In [ ]:
data = b"N"
if not isinstance(data, ctypes._CData):
    raise TypeError("Invalid type for addressof")

data_ptr = ctypes.addressof(data)
c_void_ptr = ctypes.c_void_p(data_ptr)

AttributeError: module 'ctypes' has no attribute '_CData'

In [ ]:
s.ctypes.data_as(POINTER(c_double))

In [ ]:
c_void_p(b"M")

TypeError: cannot be converted to pointer

In [32]:

# Call DGGSVD routine
liblapack.dgesvd_(c_void_p(b"N"), c_void_p(b"N"), c_int(n), c_int(lda),
                    c_int(ldb), a_data, c_int(lda), s.ctypes.data_as(POINTER(c_double)),
                    u_data, c_int(lda), v.ctypes.data_as(POINTER(c_double)), c_int(lda),
                    q.ctypes.data_as(POINTER(c_double)), c_int(ldb))

TypeError: cannot be converted to pointer

In [35]:
target_address = None
c_void_ptr = ctypes.addressof(ctypes.c_char_p(target_address))